In [1]:
using LinearAlgebra
using DataFrames,CSV
using Plots
using Statistics
using BetaML: partition
using Flux
using Distributions

In [2]:
function fc(Rij::T, Rc::T) :: T where {T}
    #se sto oltre al cutoff restituisco 0
    if Rij>=Rc
        return 0
    else 
        arg=1-1/(1-(Rij/Rc)*(Rij/Rc))

        return exp(arg)

    end
end

fc (generic function with 1 method)

In [3]:
#non sto usando le unità di misura vere per semplificare la task
function LennardJones(r::T) :: T where {T}
    epsilon=1 #in unità di epsilon
    sigma=1 #in unità di sigma
    temp=sigma/r
    V=4*epsilon*(temp^12-temp^6)
    return V
end



LennardJones (generic function with 1 method)

In [4]:
# Definisci la funzione di perdita 

loss(model, x, y) = mean(abs2.(model(x) .- y))

loss (generic function with 1 method)

In [5]:
#definisco il mio layer personalizzato 

struct MyLayer
    W_eta::AbstractArray  # Pesi per il collegamento "eta"
    W_Fs::AbstractArray   # Pesi per il collegamento "Fs"
    cutoff::Float32       # Raggio di cutoff, lo prende come argomento il layer

end

function MyLayer(input_dim::Int, hidden_dim::Int, cutoff::Float32)
    W_eta = rand(Uniform(0.25, 1), hidden_dim, input_dim)  # Inizializza i pesi per "eta" uniformemente 
    W_Fs = rand(Uniform(0, 0.5), hidden_dim, input_dim)   # Inizializza i pesi per "Fs" uniformemente 
    MyLayer(W_eta, W_Fs,cutoff)
end

function (layer::MyLayer)(x)
    return Float32.(2 .*fc.(x,layer.cutoff).*exp.(-(x.-layer.W_Fs).*(x.-layer.W_Fs).*layer.W_eta))
end

function Base.show(io::IO, layer::MyLayer)
    println(io, "CustomLayer with two links per input neuron (eta and Fs)")
    println(io, "Weights eta: ", layer.W_eta)
    println(io, "Weights Fs: ", layer.W_Fs)
    println(io, "Raggio di cutoff: ", layer.cutoff)
end

Flux.@layer MyLayer

In [ ]:
# Prima parte: qui insegniamo al modello a fare un fit del potenziale di Lennard-Jones

# Generazione del dataset
Ndata = 1000  # Numero di dati (campioni) per il dataset

# Genera distanze casuali per simulare i valori tra due atomi di elio
R = rand(Float32,Ndata) .* 1.55f0 .+ 0.95f0  # Distanze tra due atomi (in un range realistico)

# Calcola il potenziale di Lennard-Jones per ciascuna distanza in R (Target)
Target = LennardJones.(R)  # Valori target del potenziale di Lennard-Jones

# Intervallo continuo di valori di distanza per tracciare il potenziale di Lennard-Jones reale
x = Float32.(0.95:0.01:2.6)  # Distanze campionate per la curva continua
y = LennardJones.(x)  # Calcola il potenziale teorico su queste distanze

# Suddivisione in dataset di addestramento (train), validazione (validation), e test
((x_train, tempX), (y_train, tempY)) = partition([R, Target], [0.7, 0.3])  # 70% train, 30% test+validation
((x_vali, x_test), (y_vali, y_test)) = partition([tempX, tempY], [0.5, 0.5])  # 50% del test+validation va a ciascuno

# Plot del potenziale di Lennard-Jones e del set di addestramento
plot(x, y, label="Potenziale di Lennard-Jones reale", color="blue")  # Traccia la curva teorica del potenziale
scatter!(x_train, y_train, label="train set")  # Visualizza i dati di addestramento come punti

# Etichette e titolo per il grafico
xlabel!("Distanza [σ]")  # Etichetta asse x (distanza)
ylabel!("Energia [ϵ]")   # Etichetta asse y (energia)
title!("Lennard-Jones per l'elio")  # Titolo del grafico

# Trasponi e converti i dati di input e di output in Float32 per adattarli al modello
x_train = Float32.(permutedims(x_train))  # Trasforma x_train in Float32 e lo trasforma per l'addestramento
x_test = Float32.(permutedims(x_test))    # Trasforma x_test in Float32 e lo trasforma per il test
y_train = Float32.(permutedims(y_train))  # Trasforma y_train in Float32 per l'addestramento
y_test = Float32.(permutedims(y_test))    # Trasforma y_test in Float32 per il test
x_vali = Float32.(permutedims(x_vali))    # Trasforma x_vali in Float32 per la validazione
y_vali = Float32.(permutedims(y_vali))    # Trasforma y_vali in Float32 per la validazione

println(" ")  # Stampa una riga vuota (spazio nel log)


In [ ]:
# Definizione del modello 
model = Chain(
    Dense(1, 3,tanh), 
    Dense(3,2,sinc),
    Dense(2,2,sinc),
    Dense(2,2,tanh),
    Dense(2, 1)        
)

#plotto il modello allenato sui miei punti per vedere se il gradiente è non nullo in quella zona
scatter(x_train',y_train',label="Train set",alpha=0.5,color="blue")
scatter!(x_train',model(x_train)',label="Modello non allenato")


xlabel!("Distanza [σ]")
ylabel!("Energia [ϵ]")
title!("Dati di training")

savefig("Training_points_2E.png")

In [ ]:
# Parametri di addestramento e del learning rate adattivo
epochs = 10000
best_epoch = 0
best_loss = Inf
patience = 100                # Numero di epoche senza miglioramento prima di ridurre il learning rate
factor = 0.5                 # Fattore di riduzione del learning rate
min_lr = 1e-5                # Limite inferiore per il learning rate
current_lr = 0.01           # Learning rate iniziale

opt = Flux.setup(Flux.Adam(current_lr), model)  # Configura l'ottimizzatore con il learning rate iniziale

LossTrain = zeros(Float32, epochs)
LossVali = zeros(Float32, epochs)
best_model = deepcopy(model)

# Ciclo di addestramento con learning rate adattivo
no_improvement_epochs = 0  # Conta le epoche senza miglioramenti

println(" ")  # Stampa una riga vuota (spazio nel log)

In [ ]:


for epoch in 1:epochs
    # Training in batch
    data = [(x_train, y_train)]
    Flux.train!(loss, model, data, opt)

    # Calcolo delle perdite
    LossTrain[epoch] = loss(model, x_train, y_train)
    LossVali[epoch] = loss(model, x_vali, y_vali)
    
    # Controlla se la loss di validazione è migliorata almeno delò 5%
    if LossVali[epoch] < best_loss*0.95
        best_epoch = epoch
        best_loss = LossVali[epoch]
        best_model = deepcopy(model)
        no_improvement_epochs = 0  # Reset del contatore se c'è un miglioramento
    else
        no_improvement_epochs += 1  # Incrementa il contatore se non migliora
    end

    # Riduci il learning rate se non ci sono miglioramenti per un numero di epoche pari a "patience"
    if no_improvement_epochs >= patience
        current_lr = max(current_lr * factor, min_lr)  # Riduce il learning rate, ma non scende sotto min_lr
        opt = Flux.setup(Flux.Adam(current_lr), model) # Aggiorna l'ottimizzatore con il nuovo learning rate
        no_improvement_epochs = 0  # Reset del contatore
    end
end

# Stampa delle perdite finali e della migliore epoca
println("Final Training Loss: $(loss(model, x_train, y_train)), oppure $(sqrt(loss(model, x_train, y_train))*8.52e-1 ) eV")
println("Final Validation Loss: $(loss(model, x_vali, y_vali)), oppure $(sqrt(loss(model, x_vali, y_vali))*8.52e-1 ) eV")
println("Il modello migliore è stato trovato all'epoca: $best_epoch")

# Aggiorna il modello con il miglior modello salvato
model = best_model

println(" ")

In [ ]:
LossTrain_real=sqrt.(LossTrain).* 8.52e-1 #converto in eV
LossVali_real=sqrt.(LossVali).* 8.52e-1  #converto in eV

plot(1:epochs, LossTrain_real, label="Train", yscale=:log10, ylabel="radice quadrata della Loss [meV]", xlabel="Epoca",
     title="Radice quadrata della Loss in funzione dell'epoca", color="blue", linestyle=:solid)
plot!(1:epochs, LossVali_real, label="Validation", color="red", linestyle=:dot)

# Linea per la migliore epoca
#plot!([best_epoch, best_epoch], [minimum(LossTrain), 1], label="Best Epoch", color="black", linestyle=:dash, linewidth=2)

# Salva la figura
savefig("loss_fit_2E.png")


In [ ]:
# Predizioni del modello per ciascun dataset
Train_guess = model(x_train)  # Training Set
vali_guess = model(x_vali)    # Validation Set
test_guess = model(x_test)    # Test Set

# Primo grafico: Training Set
# Crea un grafico del potenziale reale e delle predizioni del modello sul training set
p1 = plot(x, y, label="Potenziale di Lennard Jones reale", color="black", lw=2)
scatter!(p1, x_train', Train_guess', label="Predizione sul train set", alpha=0.5, color="cyan")

# Etichette e titolo del grafico per il training set
xlabel!("Distanza [σ]")
ylabel!("Energia [ϵ]")
title!("Lennard Jones dell'elio - Train Set")

# Mostra e salva il primo grafico
display(p1)
savefig("2E_LJ_train_predict.png")


# Secondo grafico: Validation Set
# Crea un grafico del potenziale reale e delle predizioni del modello sul validation set
p2 = plot(x, y, label="Potenziale di Lennard Jones reale", color="black", lw=2)
scatter!(p2, x_vali', vali_guess', label="Predizione sul validation set", color="red", alpha=0.5)

# Etichette e titolo del grafico per il validation set
xlabel!("Distanza [σ]")
ylabel!("Energia [ϵ]")
title!("Lennard Jones dell'elio - Validation Set")

# Mostra e salva il secondo grafico
display(p2)
savefig("2E_LJ_validation_predict.png")


# Terzo grafico: Test Set
# Crea un grafico del potenziale reale e delle predizioni del modello sul test set
p3 = plot(x, y, label="Potenziale di Lennard Jones reale", color="black", lw=2)
scatter!(p3, x_test', test_guess', label="Predizione sul test set", color="green", alpha=0.5)

# Etichette e titolo del grafico per il test set
xlabel!("Distanza [σ]")
ylabel!("Energia [ϵ]")
title!("Lennard Jones dell'elio - Test Set")

# Mostra e salva il terzo grafico
display(p3)
savefig("2E_LJ_test_predict.png")


In [ ]:
# Creazione del grafico per il potenziale reale
p = plot(x, y, label="Potenziale di Lennard Jones reale", color="black", lw=2)

# Predizione del modello per il set di test (o uno a tua scelta)
# Puoin scegliere il set che preferisci, qui usiamo il test set
guess = model(x')

# Aggiunta delle predizioni del modello
plot!(p, x, guess', label="Predizione del modello", color="red", lw=2, linestyle=:dash)

# Imposta la dimensione del font della legenda a 12
plot!(p, legendfont=font(12))

# Etichette e titolo
xlabel!("Distanza [σ]")
ylabel!("Energia [ϵ]")
title!("Potenziale di L-J reale e Predizione del Modello")

# Aggiunge la legenda
display(p)

# Salvataggio della figura
savefig("2E_LJ_comparison_predict.png")


Da qui in poi lavoro usando le funzioni G1 e il mio layer custom

In [ ]:
R_cutoff = 2.5f0  #raggio della sfera di cutoff

# Definizione del modello 
# Creo un'istanza del layer personalizzato

G1_Number=5  #qui scelgo quante funzioni G1 usare 
println("bro")
my_layer = MyLayer(1, G1_Number,R_cutoff) 



MyModel = Chain(
    my_layer,
    Dense(G1_Number, 4,tanh),     
    Dense(4, 3,tanh),  
    Dense(3,3,tanh),
    Dense(3, 1)        
) 
       

#anche qui controllo che il modello non abbia gradiente nullo sui miei dati
Base.show(my_layer)
scatter(x_train',y_train',label="Training set")
scatter!(x_train',MyModel(x_train)',label="modello non allenato")
xlabel!("Distanza [σ]")
ylabel!("Energia [ϵ]")
title!("Dati di training e modello non allenato")



In [ ]:
# Configura l'ottimizzatore e inizializza i parametri per il learning rate adattivo
initial_lr = 0.01
min_lr = 1e-5
lr = initial_lr  # Learning rate iniziale
opt = Flux.setup(Flux.Adam(lr), MyModel)

epochs = 5000
patience = 100        # Numero di epoche senza miglioramento per ridurre il learning rate
decay_factor = 0.5   # Fattore di riduzione del learning rate
no_improve_count = 0 # Conta le epoche senza miglioramento

best_epoch = 0
best_loss = Inf
best_model = deepcopy(MyModel)  # Per salvare il miglior modello

LossTrain = zeros(Float32, epochs)
LossVali = zeros(Float32, epochs)

println(" ")

In [ ]:
# Training
for epoch in 1:epochs

    # Training in batch
    data = [(x_train, y_train)]
    Flux.train!(loss, MyModel, data, opt)

    # Calcolo delle perdite
    LossTrain[epoch] = loss(MyModel, x_train, y_train)
    LossVali[epoch] = loss(MyModel, x_vali, y_vali)

    # Salvare il miglior modello basato sulla perdita di validazione
    if LossVali[epoch] < best_loss
        best_epoch = epoch
        best_loss = LossVali[epoch]
        best_model = deepcopy(MyModel)
        no_improve_count = 0  # Reset della conta delle epoche senza miglioramento
    else
        no_improve_count += 1
    end

    # Aggiorna il learning rate se non ci sono stati miglioramenti per "patience" epoche
    if no_improve_count >= patience
        lr = max(lr * decay_factor, min_lr)  # Riduci il learning rate ma non sotto min_lr
        opt = Flux.setup(Flux.Adam(lr), MyModel)  # Reimposta l'ottimizzatore con il nuovo learning rate
        no_improve_count = 0  # Reset della conta dopo l'aggiornamento
    end
end

# Stampa delle perdite finali e della migliore epoca
println("Final Training Loss: $(loss(MyModel, x_train, y_train)), oppure $(sqrt(loss(MyModel, x_train, y_train))*8.52e-1 ) eV")
println("Final Validation Loss: $(loss(MyModel, x_vali, y_vali)), oppure $(sqrt(loss(MyModel, x_vali, y_vali))*8.52e-1 ) eV")
println("Il modello migliore è stato trovato all'epoca: $best_epoch")

# Sovrascrivi il modello attuale con il miglior modello trovato
MyModel = best_model
println(" ")


In [ ]:
LossTrain_real=sqrt.(LossTrain).* 8.52e-1 #converto in eV
LossVali_real=sqrt.(LossVali).* 8.52e-1

plot(1:epochs, LossTrain_real, label="Train", yscale=:log10, ylabel="radice quadrata della Loss [meV]", xlabel="Epoca",
     title="Radice quadrata della Loss in funzione dell'epoca", color="blue", linestyle=:solid)
plot!(1:epochs, LossVali_real, label="Validation", color="red", linestyle=:dot)

# Linea per la migliore epoca
#plot!([best_epoch, best_epoch], [minimum(LossTrain), 1], label="Best Epoch", color="black", linestyle=:dash, linewidth=2)

# Salva la figura
savefig("2E_BP_loss.png")


In [ ]:
# Predizioni del modello per ciascun dataset
Train_guess = model(x_train)  # Training Set
vali_guess = model(x_vali)    # Validation Set
test_guess = model(x_test)    # Test Set

# Primo grafico: Training Set
# Crea un grafico del potenziale reale e delle predizioni del modello sul training set
p1 = plot(x, y, label="Potenziale di Lennard Jones reale", color="black", lw=2)
scatter!(p1, x_train', Train_guess', label="Predizione sul train set", alpha=0.5, color="cyan")

# Etichette e titolo del grafico per il training set
xlabel!("Distanza [σ]")
ylabel!("Energia [ϵ]")
title!("Lennard Jones dell'elio - Train Set")

# Mostra e salva il primo grafico
display(p1)
savefig("2E_BPNN_train_predict.png")


# Secondo grafico: Validation Set
# Crea un grafico del potenziale reale e delle predizioni del modello sul validation set
p2 = plot(x, y, label="Potenziale di Lennard Jones reale", color="black", lw=2)
scatter!(p2, x_vali', vali_guess', label="Predizione sul validation set", color="red", alpha=0.5)

# Etichette e titolo del grafico per il validation set
xlabel!("Distanza [σ]")
ylabel!("Energia [ϵ]")
title!("Lennard Jones dell'elio - Validation Set")

# Mostra e salva il secondo grafico
display(p2)
savefig("2E_LJ_validation_predict.png")


# Terzo grafico: Test Set
# Crea un grafico del potenziale reale e delle predizioni del modello sul test set
p3 = plot(x, y, label="Potenziale di Lennard Jones reale", color="black", lw=2)
scatter!(p3, x_test', test_guess', label="Predizione sul test set", color="green", alpha=0.5)

# Etichette e titolo del grafico per il test set
xlabel!("Distanza [σ]")
ylabel!("Energia [ϵ]")
title!("Lennard Jones dell'elio - Test Set")

# Mostra e salva il terzo grafico
display(p3)
savefig("2E_LJ_test_predict.png")


In [ ]:
using Plots

# Creazione del grafico per il potenziale reale
p = plot(x, y, label="Potenziale di Lennard Jones reale", color="black", lw=2)

# Predizione del modello per il set di test (o uno a tua scelta)
# Puoi scegliere il set che preferisci, qui usiamo il test set
guess = MyModel(x')

# Aggiunta delle predizioni del modello
plot!(p, x, guess', label="Predizione del modello", color="red", lw=4, linestyle=:dot)

# Imposta la dimensione del font della legenda a 12
plot!(p, legendfont=font(12))

xlabel!("Distanza [σ]")
ylabel!("Energia [ϵ]")
title!("Potenziale di L-J e Predizione del Modello con ReLU")
display(p)

# Salvataggio della figura in alta qualità
# Specifica la dimensione dell'immagine
gr(size=(2000, 1200))  # Imposta una dimensione più grande per l'immagine
savefig("2E_LJ_comparison_predict_BPNN.png")
